In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [12]:
df_2022_01 = pd.read_parquet('./data/yellow_tripdata_2022-01.parquet')

In [13]:
df_2022_01.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [14]:
# Question 1: Number of columns in Jan 2022 Yellow Taxi Trip data
df_2022_01.shape 

(2463931, 19)

In [15]:
# Question 2:  Standard deviation of the trips duration in Jan 2022 Yellow Taxi Trip data
df_2022_01['duration'] = df_2022_01.tpep_dropoff_datetime - df_2022_01.tpep_pickup_datetime
df_2022_01.duration = df_2022_01.duration.apply(lambda td: td.total_seconds() / 60)
df_2022_01['duration'].std()

46.44530513776499

In [16]:
# Question 3: Fraction of the records left after dropping the outliers
df_2022_01_no_outliers = df_2022_01[(df_2022_01.duration >= 1) & (df_2022_01.duration <= 60)].copy()
len(df_2022_01_no_outliers) / len(df_2022_01)

0.9827547930522406

In [19]:
# Question 4: Dimensionality after OHE
categorical = ['PULocationID', 'DOLocationID']
df_2022_01_no_outliers[categorical] = df_2022_01_no_outliers[categorical].astype(str)
train_dicts = df_2022_01_no_outliers[categorical].to_dict(orient='records')

dv = DictVectorizer()

X_train = dv.fit_transform(train_dicts)

X_train.shape

(2421440, 515)

In [20]:
# Question 5: RMSE on train
target = 'duration'
y_train = df_2022_01_no_outliers[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

6.986190814952337

In [21]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    features = ['PULocationID', 'DOLocationID']
    df[features] = df[features].astype(str)

    return df

In [24]:
# Question 6: RMSE on validation
df_val = read_dataframe('./data/yellow_tripdata_2022-02.parquet')

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
target = 'duration'
y_train = df_2022_01_no_outliers[target].values
y_val = df_val[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.786407163179794